In [94]:
import numpy as np
import pandas as pd
import os
from task1 import create_csv1
import codecs

In [97]:
def create_dataframe(name_class_good: str, name_class_bad: str, name_annatation: str):
    create_csv1(name_class_good, name_class_bad, name_annatation)
    df = pd.read_csv(name_annatation, encoding = 'utf-16', delimiter=';',names=['absolut_path', 'otnos_path', 'class_name'])
    df = df.drop(['absolut_path', 'otnos_path'], axis=1)
    df['review'] = ''
    df['number_words'] = ''
    i = 0
    for file_dataset in os.listdir("dataset"):
        path_p = os.path.join("dataset", file_dataset)
        class_name = os.listdir(path_p)
        for elem in class_name:
            name = os.path.join(os.path.join("dataset", file_dataset ), elem)
            f = codecs.open(name, 'r', 'utf-8')
            df.loc[[i], 'review'] = f.read()
            print(f.read())
            # for j in f.read():
            #     print(j)
            #     if j == ' ' or j == '!' or j == ',' or j == '.' or j == ':' or j == ';' or j == '?' or j =='\n':
            #         count += 1
            # print(count)
            f.close()
            i += 1
    print(df)

In [98]:
if __name__ == '__main__':
    create_dataframe("good", "bad",  "annotation")